# Emulating NEB Tm Calculator

The purpose of this notebook is to attempt to replicate the behaviour of the [NEB Tm Calculator](https://tmcalculator.neb.com/#!/batch) version 1.16.5.

The NEB Tm Calculator comes with four default primers. We use these in batch mode with standard Taq polymerase and standard buffer.
Standard primer conentration.

## Settings:

![](settings1.png)

## Results:

![](settings2.png)

The results were extracted to a markdown table:


In [7]:
import Bio
Bio.__version__

'1.83'

In [8]:
primers = """\
P1fwd AGCGGATAACAATTTCACACAGGA
P1rev GTAAAACGACGGCCAGT
P3fwd AGCGGATAAGGGCAATTTCAC
P3rev GTAAAACGACGGCCA
""".splitlines()

NEBtms = (58, 54, 57, 50) # from NEB Tm Calculator results in image above

In [9]:
from Bio.SeqUtils import MeltingTemp as mt

In [10]:
for p, NEBtm in zip(primers, NEBtms):

    name, seq = p.split()
    
    tm = mt.Tm_NN(p, 
                  nn_table=mt.DNA_NN3,
                  Na=0,          # mM
                  K=50,          # mM
                  Tris=10,       # mM
                  # Mg=1.5,        # mM
                  dnac1=200,     # nM
                  dnac2=0,       # nM
                  dNTPs=0.2*4,   # mM
                  saltcorr=7)
    
    print(name, NEBtm, tm, tm - NEBtm)

P1fwd 58 57.614086700105645 -0.3859132998943551
P1rev 54 53.96469180796623 -0.03530819203376723
P3fwd 57 56.948297500718695 -0.05170249928130488
P3rev 50 50.361977364511176 0.36197736451117635


In [23]:
import csv
primerlist = []
with open('tmcalc_batch.txt', newline='') as csvfile:
    neb_file_reader = csv.reader(csvfile, delimiter='\t')
    for row in neb_file_reader:
        name, sequence, nebtm, *rest = row
        primerlist.append([name, sequence, nebtm])
print(len(primerlist))

200


In [28]:
import math

tmlist = []

for name, seq, neb in primerlist:
    bp = mt.Tm_NN(seq, 
              nn_table=mt.DNA_NN3,
              Na=0,          # mM
              K=50,          # mM
              Tris=10,       # mM
              #Mg=1.5,       # mM
              dnac1=200,     # nM
              dnac2=0,       # nM
              dNTPs=0.2*4,   # mM
              saltcorr=7)
    tmlist.append((name, int(neb), bp))
    if intneb != round(bp):
        print(name, seq, neb, round(bp))

﻿p101 AATTAATTTATGTATAAC 32 31
p113 GGCTTTTAGACAAACGTA 48 47
p119 GAATTTTGGTTTACCGCT 49 48
p34 CCCTCCGGTTTAATCTAC 51 50
p171 CAACGTAACGCCACAAAC 54 53
p52 TTATAGTGAGCCGAGGGC 56 55
p132 AGTGCATCCGGTCAGTCG 60 59
p143 GGGCAGGAGGGTGGTCCA 66 65


8/200 primers round to 1C more ore less than the NEB Tm calculator.